In [1]:
import numpy as np
import ascends as asc
import keras

Using TensorFlow backend.


In [ ]:
# Adding some more data for this particular example

def add_reference_alloy_data(x_train, header_x, stress_key = 'Stress'):
    
    for i in range(0, len(header_x)):
        if(header_x[i]==stress_key):
            idx_of_stress = i
    
    # experimental code
    x_train_update = []

    y_1 = []
    y_2 = []
    
    for i in range(0,len(x_train)):

        x = x_train[i]
        y = y_train[i][0]
        Stress = x[idx_of_stress]

        # data came fromTP347HFG-NF709, TP347HFG reference alloys
        x_update = list(x)
        if(Stress) ==70:
            x_update+=[24563.25,24126]
        elif(Stress) ==100:
            x_update+=[23592,23174]
        elif(Stress) ==130:
            x_update+=[22812.5,22553]
        elif(Stress) ==170:
            x_update+=[22198,22050.5]
        elif(Stress) ==200:
            x_update+=[21612.5,21276]
        elif(Stress) ==250:
            x_update+=[20899.25,20562]
        elif(Stress) ==300:
            x_update+=[20317.75,19825.5]

        if(x_update[-1]<y):
            y_1.append([1])
        else:
            y_1.append([0])

        if(x_update[-2]<y):
            y_2.append([1])
        else:
            y_2.append([0])

        x_train_update.append(x_update)

    x_train_update = np.array(x_train_update)
    x_train = x_train_update
    y_1 = np.array(y_1)
    y_2 = np.array(y_2)
    
    header_x = np.append(header_x, ['TP347HFG-NF709', 'TP347HFG'])

    return x_train, header_x, y_1, y_2

In [ ]:
# Loading a config file from
config_file = 'config/creep_more_features.properties'

# Now all config parameters are set 
config, csv_file, cols_to_remove, target_col, scaler_option, path_to_mine_jar, \
output_dir, num_of_features, num_of_folds, num_of_sets, model_parameters = asc.load_cfg(config_file)

print(model_parameters)

In [ ]:
# load data from csv file
data_df, x_train, y_train, header_x, header_y = asc.data_load_shuffle(csv_file, None, cols_to_remove, target_col)

# enrich data (optional), in this example, we added two more columns by using a custom function
x_train, header_x, y_1, y_2 = add_reference_alloy_data(x_train, header_x, stress_key='Stress')

In [ ]:
# performing correlation analysis
correlation_rank = asc.corr_analysis(data_df, target_col, csv_file, path_to_mine_jar, output_dir, show_charts=False, top_k=30)

In [ ]:
# Model Training
model_type = 'RF' # model type can be 'LR','RF','NN','KR','BR','SVM'
model = asc.define_model_regression(model_type, num_of_features, model_parameters, x_header_size = x_train.shape[1])

# Train and Predict
predictions, actual_values = asc.train_and_predict(model, x_train, y_train, scaler_option, num_of_folds=num_of_folds)

In [ ]:
# Evaluation (k-fold cross validation)
RMSE, R2 = asc.evaluate(predictions, actual_values)
asc.show_comparison_chart(predictions, actual_values)
print("* not tuned (%s)\t RMSE = %8.3f, R2 = %8.3f"%(model_type, RMSE, R2))

In [ ]:
# tuning hyper parameters
tuned_parameters = asc.hyperparameter_tuning(model_type, x_train, y_train
                                             , num_of_folds, scaler_option
                                           , n_iter=2000, random_state=0, verbose=1)

In [ ]:
# model training (with tuned parameters)
model = asc.define_model_regression(model_type, num_of_features, model_parameters = tuned_parameters, x_header_size = x_train.shape[1])

# Train and Predict
predictions, actual_values = asc.train_and_predict(model, x_train, y_train, scaler_option, num_of_folds=num_of_folds)

In [ ]:
# Evaluation (k-fold cross validation) after hyperparameter tuning
RMSE, R2 = asc.evaluate(predictions, actual_values)
asc.show_comparison_chart(predictions, actual_values)
print("* tuned (%s)\t RMSE = %8.3f, R2 = %8.3f"%(model_type, RMSE, R2))

In [ ]:
# train and save
model = asc.define_model_regression(model_type, num_of_features, model_parameters=tuned_parameters, x_header_size = x_train.shape[1])

asc.train_and_save(model, model_type+'-model-tuned-tag', model_type
                   , input_cols=header_x, target_col=header_y
                   , x_train=x_train, y_train=y_train, scaler_option=scaler_option, path_to_save = '.')

In [ ]:
# hyperparameter tuning and show how it performs

for model_type in ['RF','SVM','KR','BR','RF']:
    tuned_parameters = asc.hyperparameter_tuning(model_type, x_train, y_train
                                             , num_of_folds, scaler_option
                                             , n_iter=2000, random_state=0, verbose=1)
    # prediction with the tuned parameter
    model = asc.define_model_regression(model_type, num_of_features, tuned_parameters, x_header_size = x_train.shape[1])
    predictions, actual_values = asc.train_and_predict(model, x_train, y_train, scaler_option, num_of_folds=num_of_folds)
    RMSE, R2 = asc.evaluate(predictions, actual_values)
    print("* tuned (%s)\t RMSE = %8.3f, R2 = %8.3f"%(model_type, RMSE, R2))
    asc.show_comparison_chart(predictions, actual_values)


In [ ]:
from sklearn.preprocessing import StandardScaler
from matplotlib import pyplot as plt
layer = 4
params = [158, 1, 65, 179]
dropout = 0.0119
l_2 = 0.0481
lr=0.000262
epochs = 1000
batch_size = 2
num_of_folds = 5
j = 0

# Splitting the data for k-fold cross validation
x_trains, y_trains, x_tests, y_tests = asc.split_data(x_train, y_train, num_of_folds=num_of_folds)

# defining a model
model = asc.net_define(params = params, layer_n = layer, input_size = x_trains[j].shape[1], dropout=dropout, l_2=l_2)
model.summary()

# choose your scaler
scale = StandardScaler()
x_train_ = scale.fit_transform(x_trains[j])

# This is the change
x_test_ = scale.transform(x_tests[j])

# evaluating a model
print('Training initiated ..')

optimizer = keras.optimizers.Adam(lr=lr)

score, history = asc.evaluate_net(model, epochs=epochs, batch_size=batch_size, x_train = x_train_, y_train = y_trains[0], x_test = x_test_, y_test = y_tests[0], verbose = 0, optimizer=optimizer)
plt.plot(history.history['mean_absolute_percentage_error'])
plt.show()

print('Model Test mean_absolute_percentage_error:', score[1])

In [ ]:
# visualize the prediction
asc.show_comparison_chart_model_x(model, x_test_, y_tests[0])

In [ ]:
layer = None
params = None
dropout = 0.0119
l_2 = 0.0481
lr = 0.000262
epochs = 1000
batch_size = 2
num_of_folds = 5
tries = 1000
neuron_max = 256
batch_size_max = 5
layer_min = 4
layer_max = 4
dropout_max=0.02

asc.net_tuning(tries = tries, lr = lr, x_trains = x_trains, y_trains = y_trains, x_tests = x_tests, y_tests = y_tests, layer = layer, params=params, epochs=epochs, batch_size=batch_size, dropout=dropout, l_2 = l_2, neuron_max=neuron_max, batch_size_max=batch_size_max, layer_min = layer_min, layer_max=layer_max, dropout_max=dropout_max)